In [1]:
import numpy as np
import pandas as pd
import random, string, re
from Bio import Seq, SeqIO, SeqRecord

In [2]:
bases = ['A', 'C', 'T', 'G']

In [3]:
np.random.randint(0, 3, 100)

array([1, 1, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 2, 2, 1, 1, 1,
       2, 0, 2, 0, 1, 1, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 2, 1, 1, 2, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 2, 1, 1,
       2, 1, 2, 2, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2,
       0, 2, 0, 0, 2, 2, 0, 1, 1, 0, 2, 1])

In [4]:
def make_sequence(length):
    return "".join([bases[_ind] for _ind in np.random.randint(0, 3, length)])

In [5]:
def make_gene_exon_intron_list(exon_intron_length_list):
    exon_intron_list = []
    for _i in range(0, len(exon_intron_length_list)):
        target_length = exon_intron_length_list[_i]
        if _i % 2 == 0:
            exon_intron_list.append(make_sequence(target_length))
        else:
            exon_intron_list.append('GT' + make_sequence(target_length - 4) + 'AG')
    return exon_intron_list

Make some genes

In [6]:
gene_1_list = make_gene_exon_intron_list([100,20,100,20,100])
gene_2_list = make_gene_exon_intron_list([50,20,50])
gene_1_transcript_1_sequence = gene_1_list[0] + gene_1_list[2]
gene_1_transcript_2_sequence = gene_1_list[2] + gene_1_list[4]
gene_2_transcript_1_sequence = gene_2_list[0] + gene_2_list[2]

We make a 200 bp genome sequence, with 20 bases of random nucleotides on either side of the gene. The sequence is also written as a fasta file.

In [26]:
genome_sequence = make_sequence(20) + "".join(gene_1_list) + make_sequence(20) + "".join(gene_2_list) + make_sequence(20)
with open('test.fa', 'w') as fh:
    SeqIO.write(SeqRecord.SeqRecord(seq = Seq.Seq(genome_sequence), id = 'chr1', description = ''), format='fasta', handle = fh)


In [27]:
genome_sequence

'TCCAAACAACACTACCCACTCTATCACCACTCCACACCACCTATATTTTCTAATATCCTAAACCAACACCTCCCCTTTACCATTTATACCCTCATCACATAATAATCCCACTCAAACCTAGTCCCAAATATAATATCCAGCTATTCCTCTTTTCTCAAACAAATCCAAACTTCTACTATTTATTCAACCCATCCTCCCTATAACTTCCTAAAACTTTTACATACTAAAAACTCATCACCCGTTATCACACCACAACTTAGTTATACTATCAAATAAACACTCCCAATTTTAAACCCCCTATATACTAAAACCCACTTCTCATTTTCCTCATCAAAAATATTACCCCTATCATAACTCTACTAATCACATCTTTACACCTTTACCCCCTCACACAATCTTTATAACTCCCATCCTACTCTTTTCCCTAACCGTTTCCCAATCACTCCTCAGAACCTTCCTTCACCTAAACTCCTCCCACCCATCACATTAACCTAACCCACTAATTACAAATCCCACCTAA'

Make a sample gtf and write it to file

In [28]:
import csv
list_for_gft_df = [['test', 'FOO', 'gene', 21, 360, '.', '+', '.', 'gene_id "ENSG_test1";']]
list_for_gft_df.append(['chr1', 'test', 'transcript', 21, 240, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_1";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 21, 120, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_1";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 141, 240, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_1";'])
list_for_gft_df.append(['chr1', 'test', 'transcript', 141, 360, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_2";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 141, 240, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_2";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 261, 360, '.', '+', '.', 'gene_id "ENSG_test1"; transcript_id "ENST_test1_2";'])
list_for_gft_df.append(['chr1', 'test', 'gene', 381, 500, '.', '+', '.', 'gene_id "ENSG_test2";'])
list_for_gft_df.append(['chr1', 'test', 'transcript', 381, 500, '.', '+', '.', 'gene_id "ENSG_test2"; transcript_id "ENST_test2_1";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 381, 430, '.', '+', '.', 'gene_id "ENSG_test2"; transcript_id "ENST_test2_1";'])
list_for_gft_df.append(['chr1', 'test', 'exon', 451, 500, '.', '+', '.', 'gene_id "ENSG_test2"; transcript_id "ENST_test2_1";'])
pd.DataFrame(list_for_gft_df).to_csv("test.gtf", sep = "\t", header = False, index = False, quoting = csv.QUOTE_NONE)#quotechar = '')#, doublequote = False)

Make a STAR genome index

In [29]:
! STAR --runThreadN 1 --runMode genomeGenerate --genomeSAindexNbases 10 --genomeDir ./STAR_GENOME --genomeFastaFiles ./test.fa --sjdbGTFfile ./test.gtf --sjdbOverhang 35

	STAR --runThreadN 1 --runMode genomeGenerate --genomeSAindexNbases 10 --genomeDir ./STAR_GENOME --genomeFastaFiles ./test.fa --sjdbGTFfile ./test.gtf --sjdbOverhang 35
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Jul 12 16:12:23 ..... started STAR run
Jul 12 16:12:23 ... starting to generate Genome files
Jul 12 16:12:23 ..... processing annotations GTF
!!!!! WARNING: --genomeSAindexNbases 10 is too large for the genome size=520, which may cause seg-fault at the mapping step. Re-run genome generation with recommended --genomeSAindexNbases 3
Jul 12 16:12:23 ... starting to sort Suffix Array. This may take a long time...
Jul 12 16:12:23 ... sorting Suffix Array chunks and saving them to disk...
Jul 12 16:12:23 ... loading chunks from disk, packing SA...
Jul 12 16:12:23 ... finished generating suffix array
Jul 12 16:12:23 ... generating Suffix Array index
Jul 12 16:12:23 ... completed Suffix Array index
Jul 12 16:12:23 ...

In [30]:
def mutate_dna(dna_str, nm = 1):
    inds_list = [random.randrange(0,len(dna_str)) for _i in 'a'*nm]
    new_seq_list = list(dna_str)
    for _ind in inds_list:
        current_base = dna_str[_ind]
        new_base = bases[random.randrange(0,4)]
        while new_base == current_base:
            new_base = bases[random.randrange(0,4)]
        new_seq_list[_ind] = new_base
    return ''.join(new_seq_list)
    

Make some reads and write them to fastq files

In [35]:
def make_paired_reads_and_write_to_fastq_file(cell_barcode, umi, transcript_seq, read1_fh, read2_fh, read_length = 70, bc_nm = 1, umi_nm = 1):
    read_id = 'TESTREAD_' + ''.join([random.choice(string.ascii_uppercase + string.digits) for _ind in range(8)])
    read_1_str = mutate_dna(cell_barcode, bc_nm) + mutate_dna(umi, umi_nm) + 'TTTTTTTTTT'
    r1_seqrec = SeqRecord.SeqRecord(id = read_id, description = '', seq = Seq.Seq(read_1_str))
    r1_seqrec.letter_annotations['phred_quality'] = [30 for i in range(len(read_1_str))]
    SeqIO.write(r1_seqrec, format = 'fastq', handle = read1_fh)
    start_ind = random.randint(0, len(transcript_seq) - read_length)
    read_2_str = transcript_seq[start_ind:(start_ind + read_length)]
    r2_seqrec = SeqRecord.SeqRecord(id = read_id, description = '', seq = Seq.Seq(read_2_str))
    r2_seqrec.letter_annotations['phred_quality'] = [30 for i in range(read_length)]
    SeqIO.write(r2_seqrec, format = 'fastq', handle = read2_fh)

In [36]:
cell_1_barcode = 'AGATCG'
cell_2_barcode = 'CGTAGA'
umi_1 = 'ATCCG'
umi_2 = 'TAGGT'
umi_3 = 'CCTAA'

In [37]:
r1_fh = open("./test_R1.fastq", 'w')
r2_fh = open("./test_R2.fastq", 'w')

#cell 1
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_1, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_1, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_1, gene_1_transcript_2_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_1, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_1, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_1_transcript_2_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_3, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_3, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_3, gene_1_transcript_2_sequence, r1_fh, r2_fh, umi_nm = 0)

make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_3, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_1_barcode, umi_3, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)

#cell 2

make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_3, gene_1_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_1, gene_1_transcript_2_sequence, r1_fh, r2_fh, umi_nm = 0)

make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_1, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_2, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)
make_paired_reads_and_write_to_fastq_file(cell_2_barcode, umi_3, gene_2_transcript_1_sequence, r1_fh, r2_fh, umi_nm = 0)

r1_fh.close()
r2_fh.close()